# Lesson 2: Light Curves, Quality Flags, and Exoplanets

## Learning Goals: 
- Filter TESS lightcurve data based on quality flags and their meaning.
- Search the MAST archive by target name, download and view a lightcurve .fits file, only as an initial data check.
- Utilize the Lightkurve package to:
   * download and view a lightcurve
   * create a periodogram to find the maximum period
   * view a folded and normalized light curve
- Understand the potential consequences for ignoring quality flags.

<!-- - Apply data quality flags to remove less than ideal data and re-plot the lightcurve. -->

## Lesson Outline:
- Query and Download TESS data from MAST to check for observations of our system.
- Use the Lightkurve package to download TESS data, create a periodogram to find the maximum period, then phase fold and plot. 
- Remove data corresponding to data quality flags, and re-plot, to confirm or refute the detection of a planet. 
<!-- - Generate a lightcurve, then phase fold to look for a planetary transit. 
- Reassess the result using quality flags: should we really be using everything? Which flags are going to be problematic for our analysis?
- Appropriately filter the data to return a valid light curve, now without a planet.
 -->

## Imports
The following cell holds the imported packages. These packages are necessary for running the rest of the cells in this notebook, and you can expect to use some of these packages almost everytime you do astronomical research. A description of each import is as follows:

* `lightkurve` for advanced processing of our light curves
* `matplotlib.pyplot` for plotting data and images
* `numpy` to handle array mathematics
* `Observations` from astroquery.mast for querying data and observations from the MAST archive

In [ ]:
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
from astroquery.mast import Observations


%matplotlib inline

#Observations.enable_cloud_dataset()

### Choosing a Target

We will be exploring the exoplanetary system around the star HD 21749. In 2019, TESS observations lead to the discovery of 2 exoplanets around HD 21749, one of which is Earth sized. HD 21749 is a K-type main sequence star, it has an estimated mass of 0.73 $M_☉$, a radius of 0.70 $R_☉$, and a luminosity of 0.20 $L_☉$.

The inner planet, HD 21749 c, is orbiting at a distance of 0.08 AU with a period of just 7.8 days. It has a radius of 1.1 R🜨 and was the first Earth-sized planet found by TESS. The outer planet, HD 21749 b, orbits the star at a distance of 0.21 AU with a period of 35.6 days. 

These planets have been confirmed, but the orbit very close to their star, so let's see if we can actually find their transits in the TESS lightcurves, while making sure to practice good data hygiene. You can find more information about HD 21749 b and c at [NASA's Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/overview/HD%2021749#planet_HD-21749-c_collapsible), 

Note: The NASA Exoplanet Archive uses a different name for this star, "GJ 143" is the same star as HD 21749. For more information about star catalogs and naming conventions, check out this page from the [International Astronomical Union](https://www.iau.org/public/themes/naming_stars/).

<!-- We can examine all the exoplanets examined by TESS on the Target Of Interest [(TOI) Catalog](https://tev.mit.edu/data/collection/193/).

The data we will be exploring today is of the star **TOI-1380**. TESS observed this star during Sectors 15 & 16, let's analyze a TESS lightcurve to try to figure out if there is a planet orbiting this star.  -->

### Query and Download TESS data

In [ ]:
obs_table = Observations.query_criteria(objectname="HD 21749", obs_collection = 'TESS', radius = "1s")

#take a look at the results of our query
obs_table.show_in_notebook()

When checking for observations of the system you are interested in, it is important to do a quick search like this and make sure there are enough data aligned with your research goal. From this table we know TESS has two FFIs and two timeseries files. Reminder: there are two types of timeseries files that TESS hosts, Target Pixel Files and Light Curve files. 

A Target Pixel file (TPF) is a "postage stamp": a tiny subsection of TESS's full field of view that is focused around a specific target. TPFs are essentially a collection of images, allowing us to precisely measure the brightness of a particular patch of the sky.

A Light Curve (LC) is a more processed version of a TPF, with target brightness already extracted. This makes it easy to create a plot, but we lose the ability to select which pixels belong to the target, and which pixels to the background.

For more information on the different data products from TESS, head over to this page of the [TESS Archive Documentation Center](https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview). 

So, we now know there are optical timeseries files of HD 21749 taken by TESS, so we can move forward. 

### Get Data Products

Now it is time to actually get the data products that we would like to analyse. For our purposes, we will be looking for "LC" files, LC stands for Light Curve and these files are pre-made light curves processed by the TESS pipeline. You can find more information about the different file types from TESS in the [TESS Data Archive Manual](https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview). 

In [ ]:
#get all data products corresponding to the TESS observations we queried
data_products = Observations.get_product_list(obs_table)

#filter data products to just retrieve science light curves from TESS
lc_data_products = Observations.filter_products(data_products,productSubGroupDescription="LC",productType = "SCIENCE",productGroupDescription = "Minimum Recommended Products")
lc_data_products.show_in_notebook()

### Downloading FITS file

File Image Transport System (FITS) is a file format designed to store, transmit, and manipulate scientific images and associated data. It is the most widely used file type in astronomical research. A FITS file consists of one or more Header + Data Units (HDUs), where the first HDU is called the primary HDU, or primary array. The primary array may be empty or contain an N-dimensional array of pixels, such as a 1-D spectrum, a 2-D image, or a 3-D data cube. 

For more info on FITS files, here is a link to the FITS official [website](https://fits.gsfc.nasa.gov/). 

Next we will be downloading the FITS file corresponding to the light curve we want to analyse, let's take a look at what is inside the file. 

In [ ]:
data = Observations.download_products(lc_data_products)["Local Path"][0]
#Take a peek at the FITS file we downloaded
fits_file = data

fits.info(fits_file)
fits_lc = fits.open(fits_file)

lc = fits_lc[1].data
lc.columns

We can see that there are three extensions to this FITS file: PRIMARY, LIGHTCURVE, AND APERTURE. We will be interested in the LIGHTCURVE extension, so we have selected that extension and called it "lc". After printing out the data columns in lc, we see that there are columns for time and flux, which we will use to plot the data. 

Additionally, we can notice that there are different types of fluxes in this FITS file. SAP and PDCSAP. 

The Simple Aperture Photometry (SAP) is the summation of all the pixel values in a pre-defined aperture as a function of time.

The Pre-search Data Conditioning SAP flux (PDCSAP) flux is the SAP flux from which long term trends have been removed using so-called Co-trending Basis Vectors (CBVs). PDCSAP flux is usually cleaner data than the SAP flux and will have fewer systematic trends. 

For more information on the levels of data processing by TESS, head over to this page of the [TESS Archive Documentation Center](https://outerspace.stsci.edu/display/TESS/2.1+Levels+of+data+processing). 

Now we can check to see what this light curve looks like. 

#### Plot raw light curves from data as a check

In [ ]:
flux = lc['SAP_FLUX'] #SAP flux column
flux_err = lc['SAP_FLUX_ERR'] #SAP flux error
time_lc = lc['TIME'] #time column
quality = lc['QUALITY'] #quality flags column

fig = plt.figure(figsize = (11,4))
fig.add_subplot(211)
plt.plot(time_lc, flux,'.', color = "red")
plt.ylabel("FLUX (e-/s)")
plt.xlabel("BJD-2457000")

So far, we have accomplished two things: 
 1. confirmed that MAST indeed has TESS observations of our target.
 2. plotted a light curve of our target as an initial check. 
     
Do you see any potential transits? Let's continue to analyze this data to confirm or deny them!

### The LightKurve package
The LightKurve package is an open source python library of functions that make analysing light curves straightforward. From the [LightKurve documentation](https://docs.lightkurve.org/): "Lightkurve offers a user-friendly way to analyze time series data on the brightness of planets, stars, and galaxies. The package is focused on supporting science with NASA’s Kepler and TESS space telescopes, but can equally be used to analyze light curves obtained by your backyard telescope."

### Phase Folding

"Phase folding" is the act of summing up the amount of flux within a certain period that corresponds to the period variation of the flux. This will make it so that our light curve is plotted as function of orbital phase instead of time. Since we know there is a planet around this star, the period we are looking for is the orbital period of the planet. In order to find the period we want to fold the data into, we will need to create a periodogram. A periodogram is a way to visualize oscillating data in the frequency domain rather than the time domain, this change of domain is done through a Fourier transform, which is illustrated by the gif below. There is a lot more information on this in the tutorial notebook [Creating Periodograms](https://spacetelescope.github.io/mast_notebooks/notebooks/Kepler/creating_periodograms/creating_periodograms.html?highlight=periodogram). We will use the LightKurve package to create the periodogram, use the periodogram to find the maximum period, and then perform the phase folding using that period. 

<img src="https://upload.wikimedia.org/wikipedia/commons/5/50/Fourier_transform_time_and_frequency_domains.gif" width = 500>

This gif was created by Lucas Vieira and you can find it on [WikimediaCommons](https://commons.wikimedia.org/wiki/File:Continuous_Fourier_transform_of_rect_and_sinc_functions.gif).

For a good visual introduction to Fourier transforms, [this video by 3Blue1Brown](https://www.youtube.com/watch?v=spUNpyF58BY) on YouTube is a great place to start. 

Now we are going to perform the phase folding, but first we have to get the target pixel file data corresponding to one of our observations. Above, under the subheading **Get Data Products**, we queried the observations from TESS of HD 21749. We can use the table we printed out there, in the column **obs_id**, to get just one of those observations, so we don't have to retrieve the same data again. In this next cell, we just use one of the obs_id from that table above and use it to query the target pixel file for that observation. 

In [ ]:
ex_id = "tess2018206045859-s0001-0000000279741379-0120-s"

#Query MAST for this TESS observation
TESS_table = Observations.query_criteria(obs_id=ex_id) 


# Get associated science products for each Observation
data_products = Observations.get_product_list(TESS_table)

#This time we want to retrieve a target pixel file (TP) instead of a light curve (LC)
tp_prod = Observations.filter_products(data_products
                                       , obs_id = ex_id    # Using the same example as above
                                       , productSubGroupDescription = "TP")

#Download FITS file
tp_path = Observations.download_products(tp_prod)["Local Path"][0]

Now, we will generate a light curve from the target pixel file we downloaded using the Lightkurve package. The input parameters **quality_bitmask="none"** and **flux_column="sap_flux"** we included here are to ensure we do not retrieve corrected data from the function, this is just for the purposes of this lesson and you can omit this in the future if you have no preference for raw data. 

In [ ]:
tp = lk.read(tp_path, quality_bitmask="none", flux_column="sap_flux")
lc = tp.to_lightcurve()
lc.scatter()

We can see from this plot that there is a gap between the two cadences, let's limit our periodogram to just the time values before that gap. 

In [ ]:
lc_1st_half = lc[lc['time'].value < 1339]

#create the periodogram
pg = lc_1st_half.to_periodogram() 
pg.plot()

Woah! There is so much signal here it's impossible to tell what period we are looking for. The odd shape of this periodogram is due to those extremely low "dips" we see in the light curve above that occur roughly every 2.5 days. Do you think these dips look like real transits? 

Since we're only looking at 12 days of data for this system, the longest period we will be able to detect is 6 days. That would normally make a good upper limit, but we can be even more specific since we identified the 2.5 day trend by eye; let's set our range between 1.5 and 3.5 days to give ourselves a bit of wiggle room. Our `oversample_factor` will help to made the periodogram look smoother and better identify our maximum peak.

In [ ]:
pg = lc_1st_half.to_periodogram(minimum_period = 1.5, maximum_period = 3.5, oversample_factor=100)
pg.plot()

<!-- # pg = lc[lc['time'].value < 1339].to_periodogram(minimum_period=0.5, maximum_period=3) -->

Great! This looks like a pretty clear signal. The next step is to phase fold the flux corresponding to the orbital period of our potential exoplanet: the maximum period we see in the periodogram. 

In [ ]:
#get the orbital period of our planet
max_period = pg.period_at_max_power
print(max_period)

Excellent, now that we have our maximum period we can fold the light curve.

In [ ]:
folded_norm_lc = lc_1st_half.fold(max_period).normalize()
phase = folded_norm_lc['time'].value
flux = folded_norm_lc['flux']

# we'll use the quality flags later
quality = folded_norm_lc['quality']

#plot the normalized and phase folded light curve
fig = plt.figure(figsize = (11,4))
fig.add_subplot(211)
plt.plot(phase, flux,'.', color = "red")
plt.ylabel("NORMALIZED FLUX")
plt.xlabel('ORBITAL PHASE')

Wow! At first glance, this looks like a pretty striking transit. 
 
However, there are a few "warning signs" here. A transiting planet should not reduce a star's light to 0; that would mean that the planet's radius is the same or larger than its parent star. It is suspicious that our potential transits appear to be doing just that. Another warning is the shape, as a real transit often has a "down" and then "up" ('u'-shape), while ours is a harsh vertical line. 

Clearly we need to investigate the data further. One useful tool is the data quality flags.


<!-- 
In our last lesson, we learned how to plot a light curve in order to search for exoplanets through the transit method. You may have noticed that some of our light curves were noisy or had a "messy" signal, in this lesson we will be learning how to improve the quality of our data and produce better light curves. The process to do this utilizes data quality flags.  -->

## What are data quality flags?

Data quality flags are an indicator in the data that tell the user whether or not the data has been corrected or is deemed poor quality. These flags are put in by the TESS team after making a statistical assessment of the data. We can use these quality flags to throw out bad data and clean up our light curves in order to get a less noisy signal and give us a better chance of finding an exoplanet transit. 

**The STScI website provides the following advice on TESS quality flags:**

"Not all of these pixels indicate that the data quality is bad. In many cases the flags simply indicate that a correction was made. For instance, bit 7 indicates that a cosmic ray was corrected on one of the pixels in the optimal aperture. If this is the only flag on this cadence, then the data is likely fine and should not be excluded from your analysis. Different science cases may warrant different usages of these flags. As a starting point, the following flags indicate that the cadence will likely be of lesser quality: 1,2,3,4,5,6,8,10,13, and 15. More specifically, a bit-wise AND with the binary number 0101001010111111 indicate cadences that are likely of lesser quality." For more information, check out the [TESS Archive Manual](https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview) . 

### What if we ignore quality flags?
We saw from Lesson 1 that if we plot the data without taking into account any data quality flags, we see very messy signal in certain parts of the light curve. This noise can drown out a possibly faint planetary transit if the "dip" in the light curve from the transit is smaller than the variation of the bad data. Coversely, if we don't remove bad data, we run the risk of misidentifying a noisy signal as a transit.



### Clean up the data
By looking at the columns of the light curve data, we can see there are quality flags for anomolies in the data that we can now remove in order to get the best light curve possible. 

There are several quality flags in place, and you can learn about all of them in the [Archive Manual](https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview). For now, we will just be removing data that corresponds to the following flags:

- Bit 1. Attitude Tweak
- Bit 2. Safe Mode
- Bit 3. Coarse Point
- Bit 4. Earth Point
- Bit 5. Argabrightening Event (Sudden brightening across the CCD.)
- Bit 6. Reaction Wheel Desaturation
- Bit 8. Manual Exclude
- Bit 10. Impulsive outlier
- Bit 12. Straylight detected
- Bit 13. Scattered light
- Bit 15. Bad Calibration

In [ ]:
bad_bits = np.array([1,2,3,4,5,6,8,10,12,13,15])
value = 0
for v in bad_bits:
    value = value + 2**(v-1)
    
bad_data = np.bitwise_and(quality, value) >= 1 
print(bad_data)

In this bad_data array, TRUE means that the data at this index is flagged (bad), and FALSE means that the data at this index is not flagged (good). So, when we want to plot everything except the bad data, we will take our array and parse it on $\sim$bad_data. The tilde sign ($\sim$) is used when we work with Boolean values, it works as a negation operator that means it is used to reverse the Boolean values, so the command ```~boolean_array``` returns only the TRUE values in that array.

### Re-plot the light curve after removing bad data

Let's recreate our phase-folded plot from above, but this time *without* the data that has been flagged as potentially poor quality.

In [ ]:
fig = plt.figure(figsize = (11,4))

fig.add_subplot(212)
plt.plot(phase[~bad_data], flux[~bad_data],'.', color = "black") 
plt.ylabel("NORMALIZED FLUX")
plt.xlabel('ORBITAL PHASE')

What's going on here? Our signal has entirely vanished. Let's compare the two datasets by overplotting them.

### Over-plot the original light to compare

Let's try plotting our original "raw" data in red, then overplotting the cleaned data in black. That way, we'll know that any red points have bad quality flags.

In [ ]:
fig = plt.figure(figsize = (11,4))
fig.add_subplot(211)
plt.plot(phase, flux,'.', label = 'Raw Flux', color = "red")
plt.plot(phase[~bad_data], flux[~bad_data],'.', label = 'Cleaned Flux', color = "black")
plt.legend()
plt.ylabel("NORMALIZED FLUX")
plt.xlabel('PHASE')

Yikes! **All** of the "transit" data is bad. What's going on here?

To find our answer, we need to look to the [TESS Data Release Notes](https://tasoc.dk/info/docs.php) from Sector 1:

```
During Sector 1 observations, the reaction wheel speeds were reset to low values every 2.5 days to improve pointing precision. Each maneuver used the thrusters to remove angular momentum from the reaction wheels. These so-called “momentum dumps” take approximately 5 minutes, during which time the spacecraft was not in Fine Pointing mode, resulting in degraded pointing stability. After completing each momentum dump maneuver, the spacecraft returned to Fine Pointing mode and nominal pointing stability returns after ∼10 minutes.

Cadences collected during these intervals have data quality flags with bit 6 set (Reaction Wheel Desaturation Event) and bit 8 set (Manual Exclude). The quality flag intervals include a ∼20 minute buffer for the fine pointing mode to resume.
```

This is a prime example of how important it is to always account for the quality flags of our data; especially when trying to draw conclusions from a visual analysis. 

## Additional Resources

Head over to the TESS Archive Documentation Center for pages on [Data Products](https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview) and [Levels of Data Processing](https://outerspace.stsci.edu/display/TESS/2.1+Levels+of+data+processing) form TESS. 
The Data Product Overview page also has information about TESS data quality flags page all about data quality flags. 

Here is the [SIMBAD page](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=HD+21749&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id) on HD 21749. 

The [Lightkurve package website](https://docs.lightkurve.org/) has a lot of documentation on the different functions available, including their page on how the default [to_periodogram()](https://docs.lightkurve.org/reference/api/lightkurve.periodogram.LombScarglePeriodogram.from_lightcurve.html#lightkurve.periodogram.LombScarglePeriodogram.from_lightcurve) function works. 

## Acknowledgements

This notebook includes data collected with the TESS mission, obtained from the MAST data archive at the Space Telescope Science Institute (STScI). Funding for the TESS mission is provided by the NASA Explorer Program. STScI is operated by the Association of Universities for Research in Astronomy, Inc., under NASA contract NAS 5–26555.

Any published work that uses Astroquery should include a citation which can be found at [this link](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION) or the BibTeX entry is available from the package itself with: `astroquery.__citation__`

### Notebook Information:
Author: Emma Lieb

Last Updated: Sep 2023
<img style=float:right; src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 